 # Implementation of data 

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
import mysql.connector
mydb=mysql.connector.connect(user='root',password='12345',database='testnovmar')
mycursor=mydb.cursor()
mycursor.execute("SELECT * FROM testdata_novmar")

# Fetch all results from the query
values = mycursor.fetchall()

# Iterate over the fetched rows and print each row
# for row in values:
#     print(row)
print(type(values))

<class 'list'>


In [3]:
columns = [
    'parameter_index', 'record_time', 'Parameter', 'Unit',
    'phaseR', 'phaseY', 'phaseB', 'phaseN',
    'currentR', 'currentY', 'currentB', 'currentN'
]

# Convert the fetched data into a DataFrame with the specified columns
df = pd.DataFrame(values, columns=columns)

# Show the DataFrame
print(df)

        parameter_index         record_time        Parameter   Unit  phaseR  \
0                     2 2023-11-01 11:35:34  DT Volt/Current  V/Amp  238.00   
1                     4 2023-11-01 11:36:43  DT Volt/Current  V/Amp  237.00   
2                     6 2023-11-01 11:37:47  DT Volt/Current  V/Amp  237.00   
3                     8 2023-11-01 11:38:52  DT Volt/Current  V/Amp  237.00   
4                    10 2023-11-01 11:40:00  DT Volt/Current  V/Amp  238.00   
...                 ...                 ...              ...    ...     ...   
110773           286858 2024-03-31 23:54:55  DT Volt/Current  V/Amp  240.00   
110774           286860 2024-03-31 23:56:00  DT Volt/Current  V/Amp  240.00   
110775           286862 2024-03-31 23:57:07  DT Volt/Current  V/Amp  240.00   
110776           286864 2024-03-31 23:58:10  DT Volt/Current  V/Amp  240.00   
110777           286866 2024-03-31 23:59:14  DT Volt/Current  V/Amp  240.00   

        phaseY  phaseB phaseN currentR currentY cur

In [4]:
n = df.copy()

In [5]:
# Converting column dtypes as per the request
# convert datetime to object
n[['phaseR', 'phaseY', 'phaseB', 'phaseN', 'currentR', 'currentY', 'currentB', 'currentN']] = n[['phaseR', 'phaseY', 'phaseB', 'phaseN', 'currentR', 'currentY', 'currentB', 'currentN']].apply(pd.to_numeric, errors='coerce')

# Check the updated dtypes
print(n.dtypes)


parameter_index             int64
record_time        datetime64[ns]
Parameter                  object
Unit                       object
phaseR                    float64
phaseY                    float64
phaseB                    float64
phaseN                    float64
currentR                  float64
currentY                  float64
currentB                  float64
currentN                  float64
dtype: object


In [6]:
df_cleaned = n.drop_duplicates(subset='record_time', keep='first')
n = df_cleaned

In [7]:
n['record_time'] = pd.to_datetime(n['record_time'])

# Calculate the time difference between consecutive 'record_time' in seconds
n['time_diff'] = n['record_time'].diff().dt.total_seconds()

In [8]:
n.loc[n['time_diff'].isna(), 'time_diff'] = 0
n

,parameter_index,record_time,Parameter,Unit,phaseR,phaseY,phaseB,phaseN,currentR,currentY,currentB,currentN,time_diff
0,2,2023-11-01 11:35:34,DT Volt/Current,V/Amp,238.0,236.0,236.0,3.0,446.62,475.96,687.86,156.48,0.0
1,4,2023-11-01 11:36:43,DT Volt/Current,V/Amp,237.0,237.0,236.0,3.0,446.62,482.48,674.82,159.74,69.0
2,6,2023-11-01 11:37:47,DT Volt/Current,V/Amp,237.0,236.0,236.0,3.0,417.28,498.78,694.38,0.00,64.0
3,8,2023-11-01 11:38:52,DT Volt/Current,V/Amp,237.0,237.0,236.0,2.0,417.28,482.48,684.60,163.00,65.0
4,10,2023-11-01 11:40:00,DT Volt/Current,V/Amp,238.0,237.0,236.0,2.0,407.50,466.18,684.60,169.52,68.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
110773,286858,2024-03-31 23:54:55,DT Volt/Current,V/Amp,240.0,237.0,240.0,3.0,655.26,730.24,801.96,202.12,78.0
110774,286860,2024-03-31 23:56:00,DT Volt/Current,V/Amp,240.0,237.0,239.0,3.0,642.22,759.58,798.70,202.12,65.0
110775,286862,2024-03-31 23:57:07,DT Volt/Current,V/Amp,240.0,237.0,239.0,3.0,658.52,753.06,805.22,176.04,67.0
110776,286864,2024-03-31 23:58:10,DT Volt/Current,V/Amp,240.0,237.0,239.0,3.0,658.52,736.76,818.26,208.64,63.0


# Calculating the phased_diff

In [9]:
import pandas as pd

# Assuming your dataset is loaded in a DataFrame named df
n['max_phase'] = n[['phaseR', 'phaseY', 'phaseB']].max(axis=1)
n['max_phase_diff'] = n['max_phase'] - n['phaseN']
n

,parameter_index,record_time,Parameter,Unit,phaseR,phaseY,phaseB,phaseN,currentR,currentY,currentB,currentN,time_diff,max_phase,max_phase_diff
0,2,2023-11-01 11:35:34,DT Volt/Current,V/Amp,238.0,236.0,236.0,3.0,446.62,475.96,687.86,156.48,0.0,238.0,235.0
1,4,2023-11-01 11:36:43,DT Volt/Current,V/Amp,237.0,237.0,236.0,3.0,446.62,482.48,674.82,159.74,69.0,237.0,234.0
2,6,2023-11-01 11:37:47,DT Volt/Current,V/Amp,237.0,236.0,236.0,3.0,417.28,498.78,694.38,0.00,64.0,237.0,234.0
3,8,2023-11-01 11:38:52,DT Volt/Current,V/Amp,237.0,237.0,236.0,2.0,417.28,482.48,684.60,163.00,65.0,237.0,235.0
4,10,2023-11-01 11:40:00,DT Volt/Current,V/Amp,238.0,237.0,236.0,2.0,407.50,466.18,684.60,169.52,68.0,238.0,236.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110773,286858,2024-03-31 23:54:55,DT Volt/Current,V/Amp,240.0,237.0,240.0,3.0,655.26,730.24,801.96,202.12,78.0,240.0,237.0
110774,286860,2024-03-31 23:56:00,DT Volt/Current,V/Amp,240.0,237.0,239.0,3.0,642.22,759.58,798.70,202.12,65.0,240.0,237.0
110775,286862,2024-03-31 23:57:07,DT Volt/Current,V/Amp,240.0,237.0,239.0,3.0,658.52,753.06,805.22,176.04,67.0,240.0,237.0
110776,286864,2024-03-31 23:58:10,DT Volt/Current,V/Amp,240.0,237.0,239.0,3.0,658.52,736.76,818.26,208.64,63.0,240.0,237.0


In [10]:
n1 = n.copy()
def classify_alert(phase_diff):
    if phase_diff > 250:
        return 'High Alert'
    elif phase_diff < 216:
        return 'Low Alert'
    elif 216 <= phase_diff < 235:
        return 'Low Warning'
    elif 235 <= phase_diff <= 250:
        return 'Normal'
    return 'Unknown'

# Apply the classify_alert function to each phase difference column
n1['max_phase_range'] = n1['max_phase_diff'].apply(classify_alert)


n1


,parameter_index,record_time,Parameter,Unit,phaseR,phaseY,phaseB,phaseN,currentR,currentY,currentB,currentN,time_diff,max_phase,max_phase_diff,max_phase_range
0,2,2023-11-01 11:35:34,DT Volt/Current,V/Amp,238.0,236.0,236.0,3.0,446.62,475.96,687.86,156.48,0.0,238.0,235.0,Normal
1,4,2023-11-01 11:36:43,DT Volt/Current,V/Amp,237.0,237.0,236.0,3.0,446.62,482.48,674.82,159.74,69.0,237.0,234.0,Low Warning
2,6,2023-11-01 11:37:47,DT Volt/Current,V/Amp,237.0,236.0,236.0,3.0,417.28,498.78,694.38,0.00,64.0,237.0,234.0,Low Warning
3,8,2023-11-01 11:38:52,DT Volt/Current,V/Amp,237.0,237.0,236.0,2.0,417.28,482.48,684.60,163.00,65.0,237.0,235.0,Normal
4,10,2023-11-01 11:40:00,DT Volt/Current,V/Amp,238.0,237.0,236.0,2.0,407.50,466.18,684.60,169.52,68.0,238.0,236.0,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110773,286858,2024-03-31 23:54:55,DT Volt/Current,V/Amp,240.0,237.0,240.0,3.0,655.26,730.24,801.96,202.12,78.0,240.0,237.0,Normal
110774,286860,2024-03-31 23:56:00,DT Volt/Current,V/Amp,240.0,237.0,239.0,3.0,642.22,759.58,798.70,202.12,65.0,240.0,237.0,Normal
110775,286862,2024-03-31 23:57:07,DT Volt/Current,V/Amp,240.0,237.0,239.0,3.0,658.52,753.06,805.22,176.04,67.0,240.0,237.0,Normal
110776,286864,2024-03-31 23:58:10,DT Volt/Current,V/Amp,240.0,237.0,239.0,3.0,658.52,736.76,818.26,208.64,63.0,240.0,237.0,Normal


In [11]:
n1.drop(columns = ["currentR","currentY","currentB","currentN"],inplace = True)

In [12]:
import pandas as pd

# Sort by 'record_time' if needed
n1 = n1.sort_values(by='record_time')

# Initialize group_id column
n1['group_id'] = 1

# Create groups using iloc for index-based access
group_id = 1
for i in range(1, len(n1)):
    if n1.iloc[i]['time_diff'] > 70:
        group_id += 1
    n1.iloc[i, n1.columns.get_loc('group_id')] = group_id

# Show the dataframe with group IDs


In [13]:
n1

,parameter_index,record_time,Parameter,Unit,phaseR,phaseY,phaseB,phaseN,time_diff,max_phase,max_phase_diff,max_phase_range,group_id
0,2,2023-11-01 11:35:34,DT Volt/Current,V/Amp,238.0,236.0,236.0,3.0,0.0,238.0,235.0,Normal,1
1,4,2023-11-01 11:36:43,DT Volt/Current,V/Amp,237.0,237.0,236.0,3.0,69.0,237.0,234.0,Low Warning,1
2,6,2023-11-01 11:37:47,DT Volt/Current,V/Amp,237.0,236.0,236.0,3.0,64.0,237.0,234.0,Low Warning,1
3,8,2023-11-01 11:38:52,DT Volt/Current,V/Amp,237.0,237.0,236.0,2.0,65.0,237.0,235.0,Normal,1
4,10,2023-11-01 11:40:00,DT Volt/Current,V/Amp,238.0,237.0,236.0,2.0,68.0,238.0,236.0,Normal,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
110773,286858,2024-03-31 23:54:55,DT Volt/Current,V/Amp,240.0,237.0,240.0,3.0,78.0,240.0,237.0,Normal,25354
110774,286860,2024-03-31 23:56:00,DT Volt/Current,V/Amp,240.0,237.0,239.0,3.0,65.0,240.0,237.0,Normal,25354
110775,286862,2024-03-31 23:57:07,DT Volt/Current,V/Amp,240.0,237.0,239.0,3.0,67.0,240.0,237.0,Normal,25354
110776,286864,2024-03-31 23:58:10,DT Volt/Current,V/Amp,240.0,237.0,239.0,3.0,63.0,240.0,237.0,Normal,25354


In [14]:
import pandas as pd

# Assuming `n1` is your dataframe

# Reset the index to ensure sequential indexing
n1.reset_index(drop=True, inplace=True)

# Initialize the group counter and group list
group = 0
groups = []

# Group by 'group_id' first
for group_id, group_data in n1.groupby('group_id'):
    # Iterate through each row in the group
    for i in range(len(group_data)):
        # Check if the 'phaserangeR' is "Low Warning"
        if group_data.iloc[i]['max_phase_range'] == "High Alert":
            if i == 0 or group_data.iloc[i - 1]['max_phase_range'] != "High Alert":
                group += 1
            groups.append(f"Group-{group}")
        else:
            groups.append(None)

# Assign the group to the dataframe
n1['Group'] = groups

# Show the result
n1


,parameter_index,record_time,Parameter,Unit,phaseR,phaseY,phaseB,phaseN,time_diff,max_phase,max_phase_diff,max_phase_range,group_id,Group
0,2,2023-11-01 11:35:34,DT Volt/Current,V/Amp,238.0,236.0,236.0,3.0,0.0,238.0,235.0,Normal,1,None
1,4,2023-11-01 11:36:43,DT Volt/Current,V/Amp,237.0,237.0,236.0,3.0,69.0,237.0,234.0,Low Warning,1,None
2,6,2023-11-01 11:37:47,DT Volt/Current,V/Amp,237.0,236.0,236.0,3.0,64.0,237.0,234.0,Low Warning,1,None
3,8,2023-11-01 11:38:52,DT Volt/Current,V/Amp,237.0,237.0,236.0,2.0,65.0,237.0,235.0,Normal,1,None
4,10,2023-11-01 11:40:00,DT Volt/Current,V/Amp,238.0,237.0,236.0,2.0,68.0,238.0,236.0,Normal,1,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110755,286858,2024-03-31 23:54:55,DT Volt/Current,V/Amp,240.0,237.0,240.0,3.0,78.0,240.0,237.0,Normal,25354,None
110756,286860,2024-03-31 23:56:00,DT Volt/Current,V/Amp,240.0,237.0,239.0,3.0,65.0,240.0,237.0,Normal,25354,None
110757,286862,2024-03-31 23:57:07,DT Volt/Current,V/Amp,240.0,237.0,239.0,3.0,67.0,240.0,237.0,Normal,25354,None
110758,286864,2024-03-31 23:58:10,DT Volt/Current,V/Amp,240.0,237.0,239.0,3.0,63.0,240.0,237.0,Normal,25354,None


In [15]:
n1_notnull = n1.copy()

In [16]:
n1 = n1_notnull.copy()

In [17]:
import pandas as pd

# Function to calculate the time difference
def calculate_time_diff(group):
    # Check if there are any "High Alert" records in the group
    high_alert_rows = group[group['max_phase_range'] == 'High Alert']
    if high_alert_rows.empty:
        # If no high alert in the group, skip the group or handle accordingly
        group['time-diff-Group'] = None  # Set time-diff to None or handle as needed
        return group

    # Find the first occurrence of "High Alert"
    high_alert_idx = high_alert_rows.index[0]
    high_alert_time = group.loc[high_alert_idx, 'record_time']
    
    # Find the first occurrence of "None" after the high alert
    none_rows = group[group['Group'].isna() & (group['record_time'] > high_alert_time)]
    if not none_rows.empty:
        none_time = none_rows.iloc[0]['record_time']
        diff_time = (none_time - high_alert_time).total_seconds()
    else:
        # If no "None" after high alert, use the last "high alert" record time
        last_high_alert_time = high_alert_rows.iloc[-1]['record_time']
        diff_time = (last_high_alert_time - high_alert_time).total_seconds()

    group['time-diff-Group'] = diff_time
    return group

# Apply the function to each group
n1_notnull['record_time'] = pd.to_datetime(n1_notnull['record_time'])  # Ensure 'record_time' is in datetime format
n1_notnull = n1_notnull.groupby('group_id', group_keys=False).apply(calculate_time_diff)

# Display the result
n1_notnull


C:\Users\ayush\AppData\Local\Temp\ipykernel_17596\3422078380.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  n1_notnull = n1_notnull.groupby('group_id', group_keys=False).apply(calculate_time_diff)
C:\Users\ayush\AppData\Local\Temp\ipykernel_17596\3422078380.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  n1_notnull = n1_notnull.groupby('group_id', group_keys=False).apply(calculate_time_diff)


,parameter_index,record_time,Parameter,Unit,phaseR,phaseY,phaseB,phaseN,time_diff,max_phase,max_phase_diff,max_phase_range,group_id,Group,time-diff-Group
0,2,2023-11-01 11:35:34,DT Volt/Current,V/Amp,238.0,236.0,236.0,3.0,0.0,238.0,235.0,Normal,1,None,NaN
1,4,2023-11-01 11:36:43,DT Volt/Current,V/Amp,237.0,237.0,236.0,3.0,69.0,237.0,234.0,Low Warning,1,None,NaN
2,6,2023-11-01 11:37:47,DT Volt/Current,V/Amp,237.0,236.0,236.0,3.0,64.0,237.0,234.0,Low Warning,1,None,NaN
3,8,2023-11-01 11:38:52,DT Volt/Current,V/Amp,237.0,237.0,236.0,2.0,65.0,237.0,235.0,Normal,1,None,NaN
4,10,2023-11-01 11:40:00,DT Volt/Current,V/Amp,238.0,237.0,236.0,2.0,68.0,238.0,236.0,Normal,1,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110755,286858,2024-03-31 23:54:55,DT Volt/Current,V/Amp,240.0,237.0,240.0,3.0,78.0,240.0,237.0,Normal,25354,None,NaN
110756,286860,2024-03-31 23:56:00,DT Volt/Current,V/Amp,240.0,237.0,239.0,3.0,65.0,240.0,237.0,Normal,25354,None,NaN
110757,286862,2024-03-31 23:57:07,DT Volt/Current,V/Amp,240.0,237.0,239.0,3.0,67.0,240.0,237.0,Normal,25354,None,NaN
110758,286864,2024-03-31 23:58:10,DT Volt/Current,V/Amp,240.0,237.0,239.0,3.0,63.0,240.0,237.0,Normal,25354,None,NaN


In [18]:
n1_notnull[n1_notnull['Group'].notna()]

,parameter_index,record_time,Parameter,Unit,phaseR,phaseY,phaseB,phaseN,time_diff,max_phase,max_phase_diff,max_phase_range,group_id,Group,time-diff-Group
3571,9781,2023-11-04 12:58:14,DT Volt/Current,V/Amp,253.0,252.0,252.0,2.0,64.0,253.0,251.0,High Alert,821,Group-1,0.0
3572,9783,2023-11-04 12:59:31,DT Volt/Current,V/Amp,253.0,253.0,253.0,2.0,77.0,253.0,251.0,High Alert,822,Group-2,69.0
3573,9785,2023-11-04 13:00:40,DT Volt/Current,V/Amp,254.0,254.0,254.0,3.0,69.0,254.0,251.0,High Alert,822,Group-2,69.0
3574,9790,2023-11-04 13:02:40,DT Volt/Current,V/Amp,255.0,254.0,253.0,3.0,120.0,255.0,252.0,High Alert,823,Group-3,262.0
3575,9792,2023-11-04 13:03:45,DT Volt/Current,V/Amp,255.0,254.0,254.0,2.0,65.0,255.0,253.0,High Alert,823,Group-3,262.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95996,255112,2024-03-19 08:53:50,DT Volt/Current,V/Amp,252.0,248.0,251.0,1.0,79.0,252.0,251.0,High Alert,22107,Group-428,64.0
96002,255125,2024-03-19 09:01:30,DT Volt/Current,V/Amp,252.0,248.0,251.0,1.0,69.0,252.0,251.0,High Alert,22108,Group-429,130.0
96003,255127,2024-03-19 09:02:35,DT Volt/Current,V/Amp,252.0,248.0,252.0,1.0,65.0,252.0,251.0,High Alert,22108,Group-429,130.0
96014,255150,2024-03-19 09:15:49,DT Volt/Current,V/Amp,252.0,247.0,250.0,1.0,66.0,252.0,251.0,High Alert,22109,Group-430,65.0


In [19]:
n1_notnull[n1_notnull['Group'] == "Group-428"]

,parameter_index,record_time,Parameter,Unit,phaseR,phaseY,phaseB,phaseN,time_diff,max_phase,max_phase_diff,max_phase_range,group_id,Group,time-diff-Group
95996,255112,2024-03-19 08:53:50,DT Volt/Current,V/Amp,252.0,248.0,251.0,1.0,79.0,252.0,251.0,High Alert,22107,Group-428,64.0


In [20]:
# Replace NaN values in the 'time-diff-Group' column with 0
n1_notnull['time-diff-Group'] = n1_notnull['time-diff-Group'].fillna(0)


In [21]:
n2_notnull = n1_notnull[n1_notnull['Group'].notna()]

In [22]:

n2_notnull['time_diff_minutes'] = n2_notnull['time-diff-Group'] / 60  # Convert to minutes

# Create the time range
n2_notnull['time_range'] = n2_notnull['time_diff_minutes'].apply(lambda x: f"{int(np.floor(x)) }-{int(np.floor(x)) + 1} minutes")

# Output the result
n2_notnull


C:\Users\ayush\AppData\Local\Temp\ipykernel_17596\806840115.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  n2_notnull['time_diff_minutes'] = n2_notnull['time-diff-Group'] / 60  # Convert to minutes
C:\Users\ayush\AppData\Local\Temp\ipykernel_17596\806840115.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  n2_notnull['time_range'] = n2_notnull['time_diff_minutes'].apply(lambda x: f"{int(np.floor(x)) }-{int(np.floor(x)) + 1} minutes")


,parameter_index,record_time,Parameter,Unit,phaseR,phaseY,phaseB,phaseN,time_diff,max_phase,max_phase_diff,max_phase_range,group_id,Group,time-diff-Group,time_diff_minutes,time_range
3571,9781,2023-11-04 12:58:14,DT Volt/Current,V/Amp,253.0,252.0,252.0,2.0,64.0,253.0,251.0,High Alert,821,Group-1,0.0,0.000000,0-1 minutes
3572,9783,2023-11-04 12:59:31,DT Volt/Current,V/Amp,253.0,253.0,253.0,2.0,77.0,253.0,251.0,High Alert,822,Group-2,69.0,1.150000,1-2 minutes
3573,9785,2023-11-04 13:00:40,DT Volt/Current,V/Amp,254.0,254.0,254.0,3.0,69.0,254.0,251.0,High Alert,822,Group-2,69.0,1.150000,1-2 minutes
3574,9790,2023-11-04 13:02:40,DT Volt/Current,V/Amp,255.0,254.0,253.0,3.0,120.0,255.0,252.0,High Alert,823,Group-3,262.0,4.366667,4-5 minutes
3575,9792,2023-11-04 13:03:45,DT Volt/Current,V/Amp,255.0,254.0,254.0,2.0,65.0,255.0,253.0,High Alert,823,Group-3,262.0,4.366667,4-5 minutes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95996,255112,2024-03-19 08:53:50,DT Volt/Current,V/Amp,252.0,248.0,251.0,1.0,79.0,252.0,251.0,High Alert,22107,Group-428,64.0,1.066667,1-2 minutes
96002,255125,2024-03-19 09:01:30,DT Volt/Current,V/Amp,252.0,248.0,251.0,1.0,69.0,252.0,251.0,High Alert,22108,Group-429,130.0,2.166667,2-3 minutes
96003,255127,2024-03-19 09:02:35,DT Volt/Current,V/Amp,252.0,248.0,252.0,1.0,65.0,252.0,251.0,High Alert,22108,Group-429,130.0,2.166667,2-3 minutes
96014,255150,2024-03-19 09:15:49,DT Volt/Current,V/Amp,252.0,247.0,250.0,1.0,66.0,252.0,251.0,High Alert,22109,Group-430,65.0,1.083333,1-2 minutes


In [23]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Initialize the Dash app
app = dash.Dash(__name__)
app.title = "Time Range and Group Analysis"

# Load or preprocess your data
# Ensure 'record_time' is datetime
n2_notnull['record_time'] = pd.to_datetime(n1_notnull['record_time'])
n2_notnull['time_range'] = n2_notnull['time_range'].astype(str)
n2_notnull['Group'] = n2_notnull['Group'].astype(str)

# Extract month-year for grouping
n2_notnull['month_year'] = n2_notnull['record_time'].dt.to_period('M')

# Overall: Count unique groups for each time range
overall_unique_groups = (
    n2_notnull.groupby('time_range')['Group']
    .nunique()
    .reset_index(name='unique_groups_count')
    .sort_values('unique_groups_count', ascending=False)
)

# Month-wise: Count unique groups for each time range, grouped by month-year
monthwise_unique_groups = (
    n2_notnull.groupby(['month_year', 'time_range'])['Group']
    .nunique()
    .reset_index(name='unique_groups_count')
)

# Get the last 6 months dynamically
last_6_months = n2_notnull['month_year'].drop_duplicates().sort_values().tail(6).tolist()

# Pre-prepare figures for overall and each of the last 6 months
figures = {"Overall": px.bar(
    overall_unique_groups,
    x='time_range',
    y='unique_groups_count',
    title='High-Alert Instances Count by Time Range',
    labels={'time_range': 'Time Range', 'unique_groups_count': 'Unique Groups Count'},
    text='unique_groups_count'
).update_layout(
    xaxis_title='Time Range',
    yaxis_title='High-Alert Instances',
    xaxis=dict(tickangle=-45),
    height=600,
    width=1000
)}

for month in last_6_months:
    month_data = monthwise_unique_groups[monthwise_unique_groups['month_year'] == month]
    month_data = month_data.sort_values('unique_groups_count', ascending=False)
    figures[str(month)] = px.bar(
        month_data,
        x='time_range',
        y='unique_groups_count',
        title=f'Unique Groups Count by Time Range - {month}',
        labels={'time_range': 'Time Range', 'unique_groups_count': 'High Alert Instances'},
        text='unique_groups_count'
    ).update_layout(
        xaxis_title='Time Range',
        yaxis_title='High Alert Instances Count',
        xaxis=dict(tickangle=-45),
        height=600,
        width=1000
    )

# Layout of the dashboard
app.layout = html.Div([
    html.H1("Dashboard: Time Range and Group Analysis", style={'text-align': 'center'}),
    
    # Dropdown for selecting month or overall
    html.Div([
        html.Label("Select Data View:", style={'font-weight': 'bold', 'text-align': 'center'}),
        dcc.Dropdown(
            id='view-dropdown',
            options=[{'label': 'Overall', 'value': 'Overall'}] +
                    [{'label': str(month), 'value': str(month)} for month in last_6_months],
            value='Overall',
            clearable=False,
            style={'width': '50%', 'margin': 'auto'}
        ),
    ], style={'margin-top': '20px', 'text-align': 'center'}),
    
    # Graph section
    html.Div([
        dcc.Graph(id='unique-group-chart', style={'height': '600px'})
    ], style={'margin-top': '20px'})
])

# Callback for updating the chart
@app.callback(
    Output('unique-group-chart', 'figure'),
    [Input('view-dropdown', 'value')]
)
def update_chart(selected_view):
    return figures[selected_view]

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True,port = 7000)


C:\Users\ayush\AppData\Local\Temp\ipykernel_17596\4073240041.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ayush\AppData\Local\Temp\ipykernel_17596\4073240041.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ayush\AppData\Local\Temp\ipykernel_17596\4073240041.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

# For Month data and its visualization in it 

In [24]:
n1

,parameter_index,record_time,Parameter,Unit,phaseR,phaseY,phaseB,phaseN,time_diff,max_phase,max_phase_diff,max_phase_range,group_id,Group
0,2,2023-11-01 11:35:34,DT Volt/Current,V/Amp,238.0,236.0,236.0,3.0,0.0,238.0,235.0,Normal,1,None
1,4,2023-11-01 11:36:43,DT Volt/Current,V/Amp,237.0,237.0,236.0,3.0,69.0,237.0,234.0,Low Warning,1,None
2,6,2023-11-01 11:37:47,DT Volt/Current,V/Amp,237.0,236.0,236.0,3.0,64.0,237.0,234.0,Low Warning,1,None
3,8,2023-11-01 11:38:52,DT Volt/Current,V/Amp,237.0,237.0,236.0,2.0,65.0,237.0,235.0,Normal,1,None
4,10,2023-11-01 11:40:00,DT Volt/Current,V/Amp,238.0,237.0,236.0,2.0,68.0,238.0,236.0,Normal,1,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110755,286858,2024-03-31 23:54:55,DT Volt/Current,V/Amp,240.0,237.0,240.0,3.0,78.0,240.0,237.0,Normal,25354,None
110756,286860,2024-03-31 23:56:00,DT Volt/Current,V/Amp,240.0,237.0,239.0,3.0,65.0,240.0,237.0,Normal,25354,None
110757,286862,2024-03-31 23:57:07,DT Volt/Current,V/Amp,240.0,237.0,239.0,3.0,67.0,240.0,237.0,Normal,25354,None
110758,286864,2024-03-31 23:58:10,DT Volt/Current,V/Amp,240.0,237.0,239.0,3.0,63.0,240.0,237.0,Normal,25354,None


In [25]:
n2_not = n2_notnull[n2_notnull['time-diff-Group'] != 0.0]

In [26]:
import pandas as pd

# Assuming your data is in a DataFrame called df
# Extract month-year as period
n2_not['month_year'] = n2_not['record_time'].dt.to_period('M')

# Select first occurrence of time_diff_minutes and time-range for each Group in each month
first_occurrences = n2_not.drop_duplicates(subset=['month_year', 'Group'], keep='first')[['month_year', 'Group', 'time_diff_minutes', 'time_range']]

# Group by month_year and calculate the required metrics
result = first_occurrences.groupby('month_year').agg(
    total_time_diff_minutes=('time_diff_minutes', 'sum'),
    most_occurred_time_range=('time_range', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    median_time_diff_minutes=('time_diff_minutes', 'median')
).reset_index()

# Add total time difference in hours
result['total_time_diff_hours'] = result['total_time_diff_minutes'] / 60

# Display the result
result


C:\Users\ayush\AppData\Local\Temp\ipykernel_17596\2339178934.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,month_year,total_time_diff_minutes,most_occurred_time_range,median_time_diff_minutes,total_time_diff_hours
0,2023-11,60.700000,1-2 minutes,1.150000,1.011667
1,2023-12,117.900000,1-2 minutes,1.116667,1.965000
2,2024-01,18.083333,0-1 minutes,0.975000,0.301389
3,2024-02,191.116667,1-2 minutes,1.116667,3.185278
4,2024-03,309.116667,1-2 minutes,1.133333,5.151944


In [27]:
import pandas as pd
import plotly.express as px

# Assuming your data is in a DataFrame called n2_not
# Extract month-year as period
n2_not['month_year'] = n2_not['record_time'].dt.to_period('M')

# Select first occurrence of time_diff_minutes and time-range for each Group in each month
first_occurrences = n2_not.drop_duplicates(subset=['month_year', 'Group'], keep='first')[['month_year', 'Group', 'time_diff_minutes', 'time_range']]

# Group by month_year and calculate the required metrics
result = first_occurrences.groupby('month_year').agg(
    total_time_diff_minutes=('time_diff_minutes', 'sum'),
    most_occurred_time_range=('time_range', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    median_time_diff_minutes=('time_diff_minutes', 'median')
).reset_index()

# Add total time difference in hours
result['total_time_diff_hours'] = result['total_time_diff_minutes'] / 60

# Extract month name for the bar chart
result['month_name'] = result['month_year'].dt.strftime('%b')

# Create an interactive bar chart using Plotly
fig = px.bar(result, 
             x='month_name', 
             y='total_time_diff_hours', 
             labels={'month_name': 'Month', 'total_time_diff_hours': 'Time (Hours) of Low-voltage Instances'},
             title='Total Time of Low-voltage Instances')

# Customize the hover data to include the median and time range
fig.update_traces(hovertemplate=(
    'Month: %{x}<br>'
    'Time (Hours) Low-voltage Instances: %{y} hours<br>'
    'Median Time Diff: %{customdata[0]} minutes<br>'
    'Most Occurred Time Range: %{customdata[1]}'
), customdata=result[['median_time_diff_minutes', 'most_occurred_time_range']])

# Show the hours above each bar
fig.update_traces(text=result['total_time_diff_hours'], textposition='outside')

# Render the plot in the browser
fig.update_layout(
    autosize=True,
    title_x=0.5  # Center the title
)

# Display the plot
fig.show(renderer='browser')

# Display the result DataFrame with necessary details
result[['month_name', 'total_time_diff_hours', 'median_time_diff_minutes', 'most_occurred_time_range']]


C:\Users\ayush\AppData\Local\Temp\ipykernel_17596\1133962614.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,month_name,total_time_diff_hours,median_time_diff_minutes,most_occurred_time_range
0,Nov,1.011667,1.150000,1-2 minutes
1,Dec,1.965000,1.116667,1-2 minutes
2,Jan,0.301389,0.975000,0-1 minutes
3,Feb,3.185278,1.116667,1-2 minutes
4,Mar,5.151944,1.133333,1-2 minutes


In [29]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Initialize the Dash app
app = dash.Dash(__name__)
app.title = "Time Range and Group Analysis"

# Load or preprocess your data
# Ensure 'record_time' is datetime
n2_notnull['record_time'] = pd.to_datetime(n1_notnull['record_time'])
n2_notnull['time_range'] = n2_notnull['time_range'].astype(str)
n2_notnull['Group'] = n2_notnull['Group'].astype(str)

# Extract month-year for grouping
n2_notnull['month_year'] = n2_notnull['record_time'].dt.to_period('M')

# Overall: Count unique groups for each time range
overall_unique_groups = (
    n2_notnull.groupby('time_range')['Group']
    .nunique()
    .reset_index(name='unique_groups_count')
    .sort_values('unique_groups_count', ascending=False)
)

# Month-wise: Count unique groups for each time range, grouped by month-year
monthwise_unique_groups = (
    n2_notnull.groupby(['month_year', 'time_range'])['Group']
    .nunique()
    .reset_index(name='unique_groups_count')
)

# Get the last 6 months dynamically
last_6_months = n2_notnull['month_year'].drop_duplicates().sort_values().tail(6).tolist()

# Pre-prepare figures for overall and each of the last 6 months
figures = {"Overall": px.bar(
    overall_unique_groups,
    x='time_range',
    y='unique_groups_count',
    title='High Alert Instances Count by Time Range',
    labels={'time_range': 'Time Range', 'unique_groups_count': 'Unique Groups Count'},
    text='unique_groups_count'
).update_layout(
    xaxis_title='Time Range',
    yaxis_title='High Alert Instances',
    xaxis=dict(tickangle=-45),
    height=600,
    width=1000
)}

for month in last_6_months:
    month_data = monthwise_unique_groups[monthwise_unique_groups['month_year'] == month]
    month_data = month_data.sort_values('unique_groups_count', ascending=False)
    figures[str(month)] = px.bar(
        month_data,
        x='time_range',
        y='unique_groups_count',
        title=f'Unique Groups Count by Time Range - {month}',
        labels={'time_range': 'Time Range', 'unique_groups_count': 'High Alert Instances'},
        text='unique_groups_count'
    ).update_layout(
        xaxis_title='Time Range',
        yaxis_title='High Alert Instances Count',
        xaxis=dict(tickangle=-45),
        height=600,
        width=1000
    )

# Layout of the dashboard
app.layout = html.Div([
    html.H1("Dashboard: Time Range and Group Analysis", style={'text-align': 'center'}),
    
    # Dropdown for selecting month or overall
    html.Div([
        html.Label("Select Data View:", style={'font-weight': 'bold', 'text-align': 'center'}),
        dcc.Dropdown(
            id='view-dropdown',
            options=[{'label': 'Overall', 'value': 'Overall'}] +
                    [{'label': str(month), 'value': str(month)} for month in last_6_months],
            value='Overall',
            clearable=False,
            style={'width': '50%', 'margin': 'auto'}
        ),
    ], style={'margin-top': '20px', 'text-align': 'center'}),
    
    # Graph section
    html.Div([
        dcc.Graph(id='unique-group-chart', style={'height': '600px'})
    ], style={'margin-top': '20px'})
])

# Callback for updating the chart
@app.callback(
    Output('unique-group-chart', 'figure'),
    [Input('view-dropdown', 'value')]
)
def update_chart(selected_view):
    return figures[selected_view]

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True,port = 9010)


C:\Users\ayush\AppData\Local\Temp\ipykernel_17596\3666530301.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ayush\AppData\Local\Temp\ipykernel_17596\3666530301.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ayush\AppData\Local\Temp\ipykernel_17596\3666530301.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

In [ ]:
import pandas as pd
import plotly.express as px

# Assuming your data is in a DataFrame called n2_not
# Extract month-year as period
n2_not['month_year'] = n2_not['record_time'].dt.to_period('M')

# Select first occurrence of time_diff_minutes and time-range for each Group in each month
first_occurrences = n2_not.drop_duplicates(subset=['month_year', 'Group'], keep='first')[['month_year', 'Group', 'time_diff_minutes', 'time_range']]

# Group by month_year and calculate the required metrics
result = first_occurrences.groupby('month_year').agg(
    total_time_diff_minutes=('time_diff_minutes', 'sum'),
    most_occurred_time_range=('time_range', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    median_time_diff_minutes=('time_diff_minutes', 'median')
).reset_index()

# Add total time difference in hours
result['total_time_diff_hours'] = result['total_time_diff_minutes'] / 60

# Extract month name for the bar chart
result['month_name'] = result['month_year'].dt.strftime('%b')

# Create an interactive bar chart using Plotly
fig = px.bar(result, 
             x='month_name', 
             y='total_time_diff_hours', 
             labels={'month_name': 'Month', 'total_time_diff_hours': 'Time (Hours) of Low-voltage Instances'},
             title='Total Time of Low-voltage Instances')

# Customize the text on the bars to show the total time, median, and most occurred time range
fig.update_traces(
    text=result['total_time_diff_hours'].round(2).astype(str) + ' hrs' + '<br>' + 
         'Median: ' + result['median_time_diff_minutes'].astype(str) + ' mins' + '<br>' + 
         'Most Occurred: ' + result['most_occurred_time_range'],
    textposition='outside',  # Place text outside the bar
)

# Render the plot in the browser
fig.update_layout(
    autosize=True,
    title_x=0.5  # Center the title
)

# Display the plot
fig.show(renderer='browser')

# Display the result DataFrame with necessary details
result[['month_name', 'total_time_diff_hours', 'median_time_diff_minutes', 'most_occurred_time_range']]


# For Week instances 

In [ ]:
import pandas as pd

# Assuming your data is in a DataFrame called 'n2_not'

# Ensure 'record_time' is in datetime format
n2_not['record_time'] = pd.to_datetime(n2_not['record_time'])

# Extract month-year as a period
n2_not['month_year'] = n2_not['record_time'].dt.to_period('M')

# Extract the week number within each year
n2_not['week_number'] = n2_not['record_time'].dt.isocalendar().week

# Select the first occurrence of 'time_diff_minutes' for each 'Group' in each 'month_year'
first_occurrences = n2_not.drop_duplicates(subset=['month_year', 'Group'], keep='first')[['month_year', 'Group', 'time_diff_minutes', 'week_number', 'time_range']]

# Group by month_year and week_number to calculate the total time_diff_minutes, most occurred time_range, and median
weekly_data = first_occurrences.groupby(['month_year', 'week_number']).agg(
    total_time_diff_minutes=('time_diff_minutes', 'sum'),
    most_occurred_time_range=('time_range', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    median_time_diff_minutes=('time_diff_minutes', 'median')
).reset_index()

# Convert time_diff_minutes to hours
weekly_data['time_diff_hours'] = weekly_data['total_time_diff_minutes'] / 60

# Display the result
weekly_data


In [ ]:
import dash
from dash import dcc, html
import pandas as pd
import plotly.graph_objects as go
import dash_bootstrap_components as dbc

# Sample Data (replace this with your actual 'n1_not' DataFrame)
# Assuming your DataFrame 'n1_not' is already available

# Ensure 'record_time' is in datetime format
n2_not['record_time'] = pd.to_datetime(n2_not['record_time'])

# Extract month-year as a period
n2_not['month_year'] = n2_not['record_time'].dt.to_period('M')

# Extract the week number within each year
n2_not['week_number'] = n2_not['record_time'].dt.isocalendar().week

# Select the first occurrence of 'time_diff_minutes' for each 'Group' in each 'month_year'
first_occurrences = n2_not.drop_duplicates(subset=['month_year', 'Group'], keep='first')[['month_year', 'Group', 'time_diff_minutes', 'week_number', 'time_range']]

# Group by month_year and week_number to calculate the total time_diff_minutes for each week
weekly_data = first_occurrences.groupby(['month_year', 'week_number']).agg(
    total_time_diff_minutes=('time_diff_minutes', 'sum'),
    median_time_diff_minutes=('time_diff_minutes', 'median'),
    most_occurred_time_range=('time_range', lambda x: x.mode().iloc[0] if not x.mode().empty else None)
).reset_index()

# Convert time_diff_minutes to hours
weekly_data['time_diff_hours'] = weekly_data['total_time_diff_minutes'] / 60

# Create a 'week_label' column for easy display in the graph
weekly_data['week_label'] = 'Week ' + weekly_data['week_number'].astype(str)

# Initialize Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define the layout of the app
app.layout = html.Div([
    html.H1("Total Hours per Week in a Month", style={'text-align': 'center'}),
    
    # Dropdown for selecting month-year
    dcc.Dropdown(
        id='month-dropdown',
        options=[{'label': str(month), 'value': str(month)} for month in weekly_data['month_year'].unique()],
        value=str(weekly_data['month_year'].unique()[0]),  # Default value (first month)
        style={'width': '50%', 'margin': 'auto'}
    ),
    
    # Graph to display the data
    dcc.Graph(id='weekly-hours-graph')
])

# Define callback to update the graph based on dropdown selection
@app.callback(
    dash.dependencies.Output('weekly-hours-graph', 'figure'),
    [dash.dependencies.Input('month-dropdown', 'value')]
)
def update_graph(selected_month):
    # Filter the data for the selected month
    selected_data = weekly_data[weekly_data['month_year'] == selected_month]
    
    # Create the plot
    fig = go.Figure()

    fig.add_trace(
        go.Bar(
            x=selected_data['week_label'],
            y=selected_data['time_diff_hours'],
            name="Total Hours per Week",
            marker_color='blue',
            hoverinfo='x+y+text',
            text=[
                f"Total Hours: {hours:.2f} hrs<br>"
                f"Median: {median:.2f} mins<br>"
                f"Most Occurred: {time_range}"
                for hours, median, time_range in zip(
                    selected_data['time_diff_hours'], 
                    selected_data['median_time_diff_minutes'], 
                    selected_data['most_occurred_time_range']
                )
            ]
        )
    )

    # Update layout to adjust aesthetics
    fig.update_layout(
        title=f"Total Hours per Week in {selected_month}",
        xaxis_title="Week",
        yaxis_title="Total Hours",
        showlegend=False,
        height=600,
        width=800,
        template="plotly_white",
        hovermode="x unified",
        margin=dict(t=50, b=50, l=50, r=50)
    )

    return fig

if __name__ == '__main__':
    app.run_server(debug=True, port=8049)


In [ ]:
import dash
from dash import dcc, html
import pandas as pd
import plotly.graph_objects as go
import dash_bootstrap_components as dbc

# Sample Data (replace this with your actual 'n1_not' DataFrame)
# Assuming your DataFrame 'n1_not' is already available

# Ensure 'record_time' is in datetime format
n2_not['record_time'] = pd.to_datetime(n2_not['record_time'])

# Extract month-year as a period
n2_not['month_year'] = n2_not['record_time'].dt.to_period('M')

# Extract the week number within each year
n2_not['week_number'] = n2_not['record_time'].dt.isocalendar().week

# Select the first occurrence of 'time_diff_minutes' for each 'Group' in each 'month_year'
first_occurrences = n2_not.drop_duplicates(subset=['month_year', 'Group'], keep='first')[['month_year', 'Group', 'time_diff_minutes', 'week_number', 'time_range']]

# Group by month_year and week_number to calculate the total time_diff_minutes for each week
weekly_data = first_occurrences.groupby(['month_year', 'week_number']).agg(
    total_time_diff_minutes=('time_diff_minutes', 'sum'),
    median_time_diff_minutes=('time_diff_minutes', 'median'),
    most_occurred_time_range=('time_range', lambda x: x.mode().iloc[0] if not x.mode().empty else None)
).reset_index()

# Convert time_diff_minutes to hours
weekly_data['time_diff_hours'] = weekly_data['total_time_diff_minutes'] / 60

# Create a 'week_label' column for easy display in the graph
weekly_data['week_label'] = 'Week ' + weekly_data['week_number'].astype(str)

# Initialize Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define the layout of the app
app.layout = html.Div([
    html.H1("Total Hours per Week in a Month", style={'text-align': 'center'}),
    
    # Dropdown for selecting month-year
    dcc.Dropdown(
        id='month-dropdown',
        options=[{'label': str(month), 'value': str(month)} for month in weekly_data['month_year'].unique()],
        value=str(weekly_data['month_year'].unique()[0]),  # Default value (first month)
        style={'width': '50%', 'margin': 'auto'}
    ),
    
    # Graph to display the data
    dcc.Graph(id='weekly-hours-graph')
])

# Define callback to update the graph based on dropdown selection
@app.callback(
    dash.dependencies.Output('weekly-hours-graph', 'figure'),
    [dash.dependencies.Input('month-dropdown', 'value')]
)
def update_graph(selected_month):
    # Filter the data for the selected month
    selected_data = weekly_data[weekly_data['month_year'] == selected_month]
    
    # Create the plot
    fig = go.Figure()

    fig.add_trace(
        go.Bar(
            x=selected_data['week_label'],
            y=selected_data['time_diff_hours'],
            name="Total Hours per Week",
            marker_color='blue',
            hoverinfo='x+y+text',
            text=[
                f"Total Hours: {hours:.2f} hrs<br>"
                f"Median: {median:.2f} mins<br>"
                f"Most Occurred: {time_range}"
                for hours, median, time_range in zip(
                    selected_data['time_diff_hours'], 
                    selected_data['median_time_diff_minutes'], 
                    selected_data['most_occurred_time_range']
                )
            ]
        )
    )

    # Update layout to adjust aesthetics
    fig.update_layout(
        title=f"Total Hours per Week in {selected_month}",
        xaxis_title="Week",
        yaxis_title="Total Hours",
        showlegend=False,
        height=600,
        width=800,
        template="plotly_white",
        hovermode="x unified",
        margin=dict(t=50, b=50, l=50, r=50)
    )

    return fig

if __name__ == '__main__':
    app.run_server(debug=True, port=8000)


# Days in a month 

In [ ]:
import pandas as pd

# Assuming your data is in a DataFrame called 'n2_not'

# Ensure 'record_time' is in datetime format
n2_not['record_time'] = pd.to_datetime(n2_not['record_time'])

# Extract month-year as a period
n2_not['month_year'] = n2_not['record_time'].dt.to_period('M')

# Extract the day of the month
n2_not['day_of_month'] = n2_not['record_time'].dt.day

# Select the first occurrence of 'time_diff_minutes' for each 'Group' in each 'month_year'
first_occurrences = n2_not.drop_duplicates(subset=['month_year', 'Group'], keep='first')[['month_year', 'Group', 'time_diff_minutes', 'day_of_month', 'time_range']]

# Group by month_year and day_of_month to calculate the total time_diff_minutes, most occurred time_range, and median
daily_data = first_occurrences.groupby(['month_year', 'day_of_month']).agg(
    total_time_diff_minutes=('time_diff_minutes', 'sum'),
    most_occurred_time_range=('time_range', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    median_time_diff_minutes=('time_diff_minutes', 'median')
).reset_index()

# Convert time_diff_minutes to hours
daily_data['time_diff_hours'] = daily_data['total_time_diff_minutes'] / 60

# Display the result
daily_data


# Bar graph between the max phase 

In [ ]:
import pandas as pd
import plotly.express as px


# Count the occurrences of each unique status in the 'max_phase_range' column
status_counts = n1['max_phase_range'].value_counts().reset_index()
status_counts.columns = ['Phase', 'Count']

# Calculate percentage for each phase
total_count = status_counts['Count'].sum()
status_counts['Percentage'] = (status_counts['Count'] / total_count * 100).round(2)

# Define custom color mapping
color_mapping = {
    'Normal': 'orange',
    'Low Warning': 'orange',
    'High Alert': 'orange',
    'Low Alert': 'orange'
}

# Combine count and percentage into a single string for display
status_counts['Text'] = status_counts.apply(
    lambda row: f"{row['Count']} ({row['Percentage']}%)", axis=1
)

# Create the bar plot using Plotly
fig = px.bar(
    status_counts, 
    x='Phase', 
    y='Count', 
    title='Count of Max-phase', 
    labels={'Count': 'Count of States'}, 
    color='Phase', 
    text='Text',  # Use the combined text with count and percentage
    color_discrete_map=color_mapping
)

# Add text labels above each bar
fig.update_traces(textposition='outside')

# Show the plot
fig.show(renderer = 'browser')


# Line-graph for max-voltage w.r.t record-time 

In [ ]:
import pandas as pd
import plotly.graph_objects as go


# Step 1: Calculate the difference between consecutive record_time values
n2['time_diff_days'] = n2['record_time'].diff().dt.total_seconds() / (24 * 3600)

# Step 2: Identify the points where the gap is more than 1 day
n2['gap_more_than_1_day'] = n2['time_diff_days'] > 1

# Step 3: Filter out rows where min_phase_diff is 0
filtered_data = n2[n2['max_phase_diff'] > 0]

# Step 4: Create the base figure
fig = go.Figure()

# Add line segments based on the thresholds
# Green for 235-250
fig.add_trace(
    go.Scatter(
        x=filtered_data.loc[(filtered_data['max_phase_diff'] >= 235) & (filtered_data['max_phase_diff'] <= 250), 'record_time'],
        y=filtered_data.loc[(filtered_data['max_phase_diff'] >= 235) & (filtered_data['max_phase_diff'] <= 250), 'max_phase_diff'],
        mode='lines',
        line=dict(color='green', width=2),
        name='235-250 (Green)'
    )
)

# Yellow for 216-235
fig.add_trace(
    go.Scatter(
        x=filtered_data.loc[(filtered_data['max_phase_diff'] >= 216) & (filtered_data['max_phase_diff'] < 235), 'record_time'],
        y=filtered_data.loc[(filtered_data['max_phase_diff'] >= 216) & (filtered_data['max_phase_diff'] < 235), 'max_phase_diff'],
        mode='lines',
        line=dict(color='yellow', width=2),
        name='216-235 (Yellow)'
    )
)

# Red for above 250
fig.add_trace(
    go.Scatter(
        x=filtered_data.loc[filtered_data['max_phase_diff'] > 250, 'record_time'],
        y=filtered_data.loc[filtered_data['max_phase_diff'] > 250, 'max_phase_diff'],
        mode='lines',
        line=dict(color='red', width=2),
        name='Above 250 (Red)'
    )
)

# Add threshold lines
thresholds = [
    {'value': 250, 'color': 'red', 'name': 'Threshold 250'},
    {'value': 216, 'color': 'red', 'name': 'Threshold 216'},
    {'value': 235, 'color': 'yellow', 'name': 'Threshold 235'},
    {'value': 240, 'color': 'green', 'name': 'Threshold 240'}
]

for threshold in thresholds:
    fig.add_trace(
        go.Scatter(
            x=filtered_data['record_time'],
            y=[threshold['value']] * len(filtered_data),
            mode='lines',
            line=dict(color=threshold['color'], dash='dash', width=1.5),
            name=threshold['name']
        )
    )

# Add markers for gaps > 1 day
gap_points = n2[n2['gap_more_than_1_day']]
fig.add_trace(
    go.Scatter(
        x=gap_points['record_time'],
        y=gap_points['max_phase_diff'],
        mode='markers',
        marker=dict(size=10, color='black', symbol='cross'),
        name='Gap > 1 Day'
    )
)

# Update layout
fig.update_layout(
    title='Max Phase Difference Over Time',
    xaxis_title='Record Time',
    yaxis_title='Min Phase Difference',
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
)

# Show the plot in a browser
fig.show()


# Monitored Record-time 

In [ ]:
import pandas as pd

# Assuming 'n1' is your DataFrame with the 'record_time' column

# Ensure 'record_time' is in datetime format
n1['record_time'] = pd.to_datetime(n1['record_time'])

# Extract month and year as a period (e.g., '2023-11')
n1['month_year'] = n1['record_time'].dt.to_period('M')

# Sort by 'group_id' and 'record_time'
n1 = n1.sort_values(by=['group_id', 'record_time'])

# Calculate the first and last record_time for each 'group_id' and 'month_year'
group_times = n1.groupby(['group_id', 'month_year'])['record_time'].agg(['min', 'max'])

# Calculate the monitored time as the difference between the max and min record_time for each group in each month
group_times['monitored_time'] = group_times['max'] - group_times['min']

# Convert monitored_time to minutes
group_times['monitored_time_minutes'] = group_times['monitored_time'].dt.total_seconds() / 60

# Group by 'month_year' and sum the monitored time for each month
monthly_monitored_time = group_times.groupby('month_year')['monitored_time_minutes'].sum() / 60  # Convert total minutes to hours

# Display the result
print("Total monitored time across all unique group_ids for each month (in hours):")
print(monthly_monitored_time)


In [30]:
import pandas as pd
import plotly.express as px

# Ensure record_time is in datetime format
n2_not['record_time'] = pd.to_datetime(n2_not['record_time'])
n1['record_time'] = pd.to_datetime(n1['record_time'])

# Extract month-year as period
n2_not['month_year'] = n2_not['record_time'].dt.to_period('M')
n1['month_year'] = n1['record_time'].dt.to_period('M')

# Calculate first occurrences from n2_not
first_occurrences = n2_not.drop_duplicates(subset=['month_year', 'Group'], keep='first')[['month_year', 'Group', 'time_diff_minutes', 'time_range']]

# Group by month_year for low-voltage stats
low_voltage_stats = first_occurrences.groupby('month_year').agg(
    total_time_diff_minutes=('time_diff_minutes', 'sum'),
    most_occurred_time_range=('time_range', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    median_time_diff_minutes=('time_diff_minutes', 'median')
).reset_index()

# Add total time in hours
low_voltage_stats['total_time_diff_hours'] = low_voltage_stats['total_time_diff_minutes'] / 60
low_voltage_stats['month_name'] = low_voltage_stats['month_year'].dt.strftime('%b')

# Calculate monitored time from n1
n1 = n1.sort_values(by=['group_id', 'record_time'])
group_times = n1.groupby(['group_id', 'month_year'])['record_time'].agg(['min', 'max'])
group_times['monitored_time'] = group_times['max'] - group_times['min']
group_times['monitored_time_minutes'] = group_times['monitored_time'].dt.total_seconds() / 60

monthly_monitored_time = group_times.groupby('month_year')['monitored_time_minutes'].sum() / 60  # Convert to hours
monitored_hours = monthly_monitored_time.reset_index()
monitored_hours.columns = ['month_year', 'monitored_hours']

# Merge monitored hours with low-voltage stats
final_result = pd.merge(low_voltage_stats, monitored_hours, on='month_year')
final_result['percentage'] = (final_result['total_time_diff_hours'] / final_result['monitored_hours']) * 100

# Visualization for low-voltage hours
fig = px.bar(final_result, 
             x='month_name', 
             y='total_time_diff_hours', 
             labels={'month_name': 'Month', 'total_time_diff_hours': 'Time (Hours) of High-Alert Instances'},
             title='Total Time of High-Alert Instances')

# Add custom text with percentages
fig.update_traces(
    text=final_result['total_time_diff_hours'].round(2).astype(str) + ' hrs' + '<br>' +
         'Percentage: ' + final_result['percentage'].round(2).astype(str) + '%' + '<br>' +
         'Median: ' + final_result['median_time_diff_minutes'].astype(str) + ' mins' + '<br>' +
         'Most Occurred: ' + final_result['most_occurred_time_range'],
    textposition='outside'
)

fig.update_layout(
    autosize=True,
    title_x=0.5
)

# Display the plot
fig.show(renderer='browser')

# Display final results
print("High-Voltage stats and monitored hours with percentages:")
print(final_result[['month_name', 'total_time_diff_hours', 'monitored_hours', 'percentage', 'median_time_diff_minutes', 'most_occurred_time_range']])


C:\Users\ayush\AppData\Local\Temp\ipykernel_17596\1810128292.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ayush\AppData\Local\Temp\ipykernel_17596\1810128292.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



High-Voltage stats and monitored hours with percentages:
  month_name  total_time_diff_hours  monitored_hours  percentage  \
0        Nov               1.011667       283.953333    0.356279   
1        Dec               1.965000       424.515000    0.462881   
2        Jan               0.301389       243.911111    0.123565   
3        Feb               3.185278        64.647222    4.927169   
4        Mar               5.151944       515.008611    1.000361   

   median_time_diff_minutes most_occurred_time_range  
0                  1.150000              1-2 minutes  
1                  1.116667              1-2 minutes  
2                  0.975000              0-1 minutes  
3                  1.116667              1-2 minutes  
4                  1.133333              1-2 minutes  


# for week 

In [ ]:
import pandas as pd

# Assuming 'n1' is your DataFrame with the 'record_time' column

# Ensure 'record_time' is in datetime format
n1['record_time'] = pd.to_datetime(n1['record_time'])

# Extract month, year, and week number (ISO calendar week)
n1['month_year'] = n1['record_time'].dt.to_period('M')
n1['week_number'] = n1['record_time'].dt.isocalendar().week

# Sort by 'group_id' and 'record_time'
n1 = n1.sort_values(by=['group_id', 'record_time'])

# Calculate the first and last record_time for each 'group_id', 'month_year', and 'week_number'
group_times = n1.groupby(['group_id', 'month_year', 'week_number'])['record_time'].agg(['min', 'max'])

# Calculate the monitored time as the difference between the max and min record_time for each group in each week
group_times['monitored_time'] = group_times['max'] - group_times['min']

# Convert monitored_time to minutes
group_times['monitored_time_minutes'] = group_times['monitored_time'].dt.total_seconds() / 60

# Group by 'month_year' and 'week_number' and sum the monitored time for each week
weekly_monitored_time = group_times.groupby(['month_year', 'week_number'])['monitored_time_minutes'].sum() / 60  # Convert total minutes to hours

# Display the result
print("Total monitored time across all unique group_ids for each week (in hours):")
print(weekly_monitored_time)


In [32]:
import pandas as pd
import plotly.graph_objects as go
from dash import dcc, html, Dash, callback, Input, Output
import dash_bootstrap_components as dbc

# Ensure 'record_time' is in datetime format
n2_not['record_time'] = pd.to_datetime(n2_not['record_time'])
n1['record_time'] = pd.to_datetime(n1['record_time'])

# Extract week number and month-year
n2_not['month_year'] = n2_not['record_time'].dt.to_period('M')
n2_not['week_number'] = n2_not['record_time'].dt.isocalendar().week

n1['month_year'] = n1['record_time'].dt.to_period('M')
n1['week_number'] = n1['record_time'].dt.isocalendar().week

# --- Low Voltage Time (from n2_not) ---
first_occurrences = n2_not.drop_duplicates(subset=['month_year', 'week_number', 'Group'], keep='first')[
    ['month_year', 'week_number', 'Group', 'time_diff_minutes', 'time_range']
]

weekly_low_voltage = first_occurrences.groupby(['month_year', 'week_number']).agg(
    total_time_diff_minutes=('time_diff_minutes', 'sum'),
    median_time_diff_minutes=('time_diff_minutes', 'median'),
    most_occurred_time_range=('time_range', lambda x: x.mode().iloc[0] if not x.mode().empty else None)
).reset_index()

# Convert total time to hours
weekly_low_voltage['time_diff_hours'] = weekly_low_voltage['total_time_diff_minutes'] / 60
weekly_low_voltage['week_label'] = 'Week ' + weekly_low_voltage['week_number'].astype(str)

# --- Monitored Time (from n1) ---
group_times = n1.groupby(['group_id', 'month_year', 'week_number'])['record_time'].agg(['min', 'max'])
group_times['monitored_time_minutes'] = (group_times['max'] - group_times['min']).dt.total_seconds() / 60

weekly_monitored_time = group_times.groupby(['month_year', 'week_number'])['monitored_time_minutes'].sum().reset_index()
weekly_monitored_time['monitored_hours'] = weekly_monitored_time['monitored_time_minutes'] / 60
weekly_monitored_time['week_label'] = 'Week ' + weekly_monitored_time['week_number'].astype(str)

# --- Merge and Calculate Percentages ---
weekly_data = pd.merge(
    weekly_low_voltage,
    weekly_monitored_time,
    on=['month_year', 'week_number', 'week_label'],
    how='outer'
)

weekly_data['percentage'] = (weekly_data['time_diff_hours'] / weekly_data['monitored_hours']) * 100
weekly_data.fillna(0, inplace=True)  # Handle missing values

# --- Dash App ---
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    html.H1("Weekly High-Voltage Time and Percentage", style={'text-align': 'center'}),
    
    # Dropdown for selecting month-year
    dcc.Dropdown(
        id='month-dropdown',
        options=[{'label': str(month), 'value': str(month)} for month in weekly_data['month_year'].unique()],
        value=str(weekly_data['month_year'].unique()[0]),  # Default to first month
        style={'width': '50%', 'margin': 'auto'}
    ),
    
    # Graph to display the weekly data
    dcc.Graph(id='weekly-hours-graph')
])

@callback(
    Output('weekly-hours-graph', 'figure'),
    Input('month-dropdown', 'value')
)
def update_graph(selected_month):
    # Filter data for the selected month
    filtered_data = weekly_data[weekly_data['month_year'] == selected_month]
    
    # Create the figure
    fig = go.Figure()

    # Bar for low-voltage hours
    fig.add_trace(
        go.Bar(
            x=filtered_data['week_label'],
            y=filtered_data['time_diff_hours'],
            name="High-Voltage Hours",
            marker_color='blue',
            hoverinfo='x+y+text',
            text=[
                f"High-Voltage Hours: {hours:.2f} hrs<br>"
                f"Percentage of Total: {percentage:.2f}%<br>"
                f"Median: {median:.2f} mins<br>"
                f"Most Occurred: {time_range}"
                for hours, percentage, median, time_range in zip(
                    filtered_data['time_diff_hours'],
                    filtered_data['percentage'],
                    filtered_data['median_time_diff_minutes'],
                    filtered_data['most_occurred_time_range']
                )
            ]
        )
    )

    # Update layout
    fig.update_layout(
        title=f"Weekly High-Voltage Time and Percentage ({selected_month})",
        xaxis_title="Week",
        yaxis_title="High-Voltage Hours",
        showlegend=False,
        template="plotly_white",
        hovermode="x unified",
        height=600,
        width=900
    )

    return fig

if __name__ == '__main__':
    app.run_server(debug=True, port=7000)


C:\Users\ayush\AppData\Local\Temp\ipykernel_17596\625682475.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ayush\AppData\Local\Temp\ipykernel_17596\625682475.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ayush\AppData\Local\Temp\ipykernel_17596\625682475.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

# For Days in a month 

In [ ]:
import pandas as pd

# Assuming 'n1' is your DataFrame with the 'record_time' column

# Ensure 'record_time' is in datetime format
n1['record_time'] = pd.to_datetime(n1['record_time'])

# Extract month, year, and day of the month
n1['month_year'] = n1['record_time'].dt.to_period('M')
n1['day_of_month'] = n1['record_time'].dt.day

# Sort by 'group_id' and 'record_time'
n1 = n1.sort_values(by=['group_id', 'record_time'])

# Calculate the first and last record_time for each 'group_id', 'month_year', and 'day_of_month'
group_times = n1.groupby(['group_id', 'month_year', 'day_of_month'])['record_time'].agg(['min', 'max'])

# Calculate the monitored time as the difference between the max and min record_time for each group in each day
group_times['monitored_time'] = group_times['max'] - group_times['min']

# Convert monitored_time to minutes
group_times['monitored_time_minutes'] = group_times['monitored_time'].dt.total_seconds() / 60

# Group by 'month_year' and 'day_of_month' and sum the monitored time for each day
daily_monitored_time = group_times.groupby(['month_year', 'day_of_month'])['monitored_time_minutes'].sum() / 60  # Convert total minutes to hours

# Display the result
print("Total monitored time across all unique group_ids for each day (in hours):")
print(daily_monitored_time)


In [35]:
import pandas as pd
import plotly.graph_objects as go
from dash import dcc, html, Dash, callback, Input, Output
import dash_bootstrap_components as dbc

# Ensure 'record_time' is in datetime format
n2_not['record_time'] = pd.to_datetime(n2_not['record_time'])
n1['record_time'] = pd.to_datetime(n1['record_time'])

# Extract month-year and day_of_month from n2_not
n2_not['month_year'] = n2_not['record_time'].dt.to_period('M')
n2_not['day_of_month'] = n2_not['record_time'].dt.day

# Select first occurrence of 'time_diff_minutes' for each 'Group' in each 'month_year'
first_occurrences = n2_not.drop_duplicates(subset=['month_year', 'Group'], keep='first')[['month_year', 'Group', 'time_diff_minutes', 'day_of_month', 'time_range']]

# Group by month_year and day_of_month to calculate the total time_diff_minutes, most occurred time_range, and median
daily_low_voltage = first_occurrences.groupby(['month_year', 'day_of_month']).agg(
    total_time_diff_minutes=('time_diff_minutes', 'sum'),
    most_occurred_time_range=('time_range', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    median_time_diff_minutes=('time_diff_minutes', 'median')
).reset_index()

# Convert total time_diff_minutes to hours
daily_low_voltage['time_diff_hours'] = daily_low_voltage['total_time_diff_minutes'] / 60

# --- Calculate Monitored Time (from n1) ---
n1['month_year'] = n1['record_time'].dt.to_period('M')
n1['day_of_month'] = n1['record_time'].dt.day

# Sort by 'group_id' and 'record_time'
n1 = n1.sort_values(by=['group_id', 'record_time'])

# Calculate the first and last record_time for each 'group_id', 'month_year', and 'day_of_month'
group_times = n1.groupby(['group_id', 'month_year', 'day_of_month'])['record_time'].agg(['min', 'max'])

# Calculate the monitored time as the difference between the max and min record_time for each group in each day
group_times['monitored_time'] = group_times['max'] - group_times['min']

# Convert monitored_time to minutes
group_times['monitored_time_minutes'] = group_times['monitored_time'].dt.total_seconds() / 60

# Group by 'month_year' and 'day_of_month' and sum the monitored time for each day
daily_monitored_time = group_times.groupby(['month_year', 'day_of_month'])['monitored_time_minutes'].sum().reset_index()

# Convert total monitored time from minutes to hours
daily_monitored_time['monitored_hours'] = daily_monitored_time['monitored_time_minutes'] / 60

# --- Merge Data ---
# Merge the daily low-voltage data with the daily monitored time data
daily_data = pd.merge(
    daily_low_voltage,
    daily_monitored_time,
    on=['month_year', 'day_of_month'],
    how='outer'
)

# Calculate the percentage of low-voltage hours over monitored hours
daily_data['percentage'] = (daily_data['time_diff_hours'] / daily_data['monitored_hours']) * 100
daily_data.fillna(0, inplace=True)  # Handle missing values

# --- Dash App ---
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    html.H1("Daily High-Voltage Time and Percentage", style={'text-align': 'center'}),
    
    # Dropdown for selecting month-year
    dcc.Dropdown(
        id='month-dropdown',
        options=[{'label': str(month), 'value': str(month)} for month in daily_data['month_year'].unique()],
        value=str(daily_data['month_year'].unique()[0]),  # Default to first month
        style={'width': '50%', 'margin': 'auto'}
    ),
    
    # Graph to display the daily data
    dcc.Graph(id='daily-hours-graph')
])

@callback(
    Output('daily-hours-graph', 'figure'),
    Input('month-dropdown', 'value')
)
def update_graph(selected_month):
    # Filter data for the selected month
    filtered_data = daily_data[daily_data['month_year'] == selected_month]
    
    # Create the figure
    fig = go.Figure()

    # Bar for low-voltage hours
    fig.add_trace(
        go.Bar(
            x=filtered_data['day_of_month'],
            y=filtered_data['time_diff_hours'],
            name="High-Voltage Hours",
            marker_color='blue',
            hoverinfo='x+y+text',
            text=[
                f"High-Voltage Hours: {hours:.2f} hrs<br>"
                f"Percentage of Total: {percentage:.2f}%<br>"
                f"Median: {median:.2f} mins<br>"
                f"Most Occurred: {time_range}"
                for hours, percentage, median, time_range in zip(
                    filtered_data['time_diff_hours'],
                    filtered_data['percentage'],
                    filtered_data['median_time_diff_minutes'],
                    filtered_data['most_occurred_time_range']
                )
            ]
        )
    )

    # Update layout
    fig.update_layout(
        title=f"Daily High-voltage Time and Percentage ({selected_month})",
        xaxis_title="Day of the Month",
        yaxis_title="High-Voltage Hours",
        showlegend=False,
        template="plotly_white",
        hovermode="x unified",
        height=600,
        width=900
    )

    return fig

if __name__ == '__main__':
    app.run_server(debug=True, port=8001)


C:\Users\ayush\AppData\Local\Temp\ipykernel_17596\4233570299.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ayush\AppData\Local\Temp\ipykernel_17596\4233570299.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ayush\AppData\Local\Temp\ipykernel_17596\4233570299.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan